In [7]:
import pickle
from multiprocessing import Pool
from glob import glob
from collections import Counter
from astropy.utils.data import get_pkg_data_filename
import pyink as pu

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import astropy.units as u
from tqdm import tqdm as tqdm
from astropy.io import fits
from astropy.table import Table
from astropy.wcs import WCS
from reproject import reproject_interp
import numpy as np
import matplotlib.pyplot as plt
import cv2

## Circle Hough Transform

In [4]:
import os
cwd = os.getcwd()
cwd

'/bigdata/users/20744696/Discovery Project/Output Object Detection'

In [ ]:
regionNum = [1,2,3,4,5,6]
lowerThres = [2,3,4,5]

for i in lowerThres:
    for j in regionNum:
        img = cv.imread(f"/bigdata/users/20744696/Discovery Project/region{j}preprocessed_EMU_image_lower{i}.png")
        output = img.copy()
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        gray = cv.medianBlur(gray, 5)
        circles = cv.HoughCircles(gray, 
                                  cv.HOUGH_GRADIENT, 
                                  1.5, 
                                  500, 
                                  param1=300, param2=10, 
                                  minRadius=5, 
                                  maxRadius=40)
        
        if circles is not None:
            detected_circles = np.uint16(np.around(circles))
            for (x, y, r) in detected_circles[0, :]:
                cv.circle(output, (x, y), r, (0, 255, 0), 2)  # Circle outline
                cv.circle(output, (x, y), 1, (0, 255, 255), 2)  # Circle center
        else:
            print("No circles were detected. Try adjusting the parameters.")
        
        # Display the image with detected circles
        plt.imshow(cv.cvtColor(output, cv.COLOR_BGR2RGB))
        plt.axis('off')
        plt.savefig(f"noCordCircleHoughTrans_output_region{j}preprocessed_EMU_image_lower{i}", dpi=500)
        plt.show()
        plt.close()

# simpleBlobDetector

In [ ]:
import math
regionNum = [1,2,3,4,5,6]
lowerThres = [2,3,4,5]

for i in lowerThres:
    for j in regionNum:
        im = cv2.imread(f"/bigdata/users/20744696/Discovery Project/region{j}preprocessed_EMU_image_lower{i}.png", cv2.IMREAD_GRAYSCALE)
        
        if im is None:
            print("Error: Could not load the image.")
        else:
            #Setup SimpleBlobDetector_create parameters
            params = cv2.SimpleBlobDetector_Params()
        
            #filter by thresholdes
            params.minThreshold = 20
            params.maxThreshold = 150
            
            #Filter by Area
            params.filterByArea = 1
            params.minArea = 2
            params.maxArea = math.pi * 10**2
        
            #filter by Circularity
            
            #Filter by inertia
            params.filterByInertia = 1
            params.minInertiaRatio = 0.2
            params.maxInertiaRatio = 0.9
        
            #Convexity
            #params.filterByConvexity = 1
            #params.minConvexity = 0.5
            #params.maxConvexity = 0.7
            
            # Set up the blob detector with default parameters
            ver = (cv2.__version__).split('.')
            if int(ver[0])<3:
                detector = cv2.SimpleBlobDetector(params)
            else:
                detector = cv2.SimpleBlobDetector_create(params)
        
            #Detect Blobs:
            keypoints = detector.detect(im)
            
            # Draw detected blobs as red circles.
            # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
            im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,255,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        
        #shows image:
        plt.imshow(cv2.cvtColor(im_with_keypoints, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.savefig(f"noCordsimpleBlob_output_region{j}_EMU_lower{i}", dpi=500)
        plt.show()
        plt.close()